In [1]:
import parmed as pmd
import numpy as np
import os
import pandas as pd

import matplotlib as mpl
import matplotlib.pyplot as plt
from matplotlib.gridspec import GridSpec
from matplotlib.colors import colorConverter
import seaborn as sns

%load_ext autoreload
%autoreload 2
%matplotlib inline

# Parameter comparisons

I find there are a different number of dihedral parameters applied to a system parameterized with GAFF v1.7 and SMIRNOFF99Frosst.

## Method 1 (use ParmEd)

In [4]:
smirnoff_prmtop = pmd.load_file("systems/a-bam-p/smirnoff/a000/hg.prmtop")
smirnoff_prmtop = smirnoff_prmtop[":4"]

gaff_prmtop = pmd.load_file("systems/a-bam-p/bgbg-tip3p/hg.topo")
gaff_prmtop = gaff_prmtop[":4"]

In [14]:
def find_dihedrals(structure):
    df = pd.DataFrame()
    for dihedral in structure.dihedrals:
        df = df.append(
            pd.DataFrame(
                {
                    "atom1": dihedral.atom1.name,
                    "atom2": dihedral.atom2.name,
                    "atom3": dihedral.atom3.name,
                    "atom4": dihedral.atom4.name,
                },
                index=[0],
            ),
            ignore_index=True,
        )
    return df

In [38]:
gaff_prmtop_dihedrals = find_dihedrals(gaff_prmtop)
smirnoff_prmtop_dihedrals = find_dihedrals(smirnoff_prmtop)

These are just labeled by *atom name* so it is safe to drop duplicates by only keeping the unique rows.

In [42]:
gaff_prmtop_dihedrals.drop_duplicates(inplace=True)
smirnoff_prmtop_dihedrals.drop_duplicates(inplace=True)

In [43]:
len(gaff_prmtop_dihedrals)

54

In [44]:
len(smirnoff_prmtop_dihedrals)

54

In [45]:
gaff_prmtop_dihedrals.sort_values(by=["atom1", "atom2", "atom3", "atom4"]).head()

,atom1,atom2,atom3,atom4
26,C1,C2,C3,C4
81,C1,C2,C3,H3
25,C1,C2,C3,O3
79,C1,C2,O2,HO2
29,C1,O5,C5,C4


In [46]:
smirnoff_prmtop_dihedrals.sort_values(by=["atom1", "atom2", "atom3", "atom4"]).head()

,atom1,atom2,atom3,atom4
10,C1,C2,C3,C4
43,C1,C2,C3,H3
9,C1,C2,C3,O3
70,C1,C2,O2,HO2
26,C1,O5,C5,C4


I think there are 54 unique dihedrals by name in a single `:MGO` residue and both GAFF v1.7 and SMIRNOFF99Frosst agree on the names. A little tricky to test for strict equality in the DataFrame because everything is a string, but I'm feeling pretty confident that this is correct.

Next, let's loop through each row, and print the dihedral in GAFF v1.7 and SMIRNOFF99Frosst.

In [139]:
for index, row in gaff_prmtop_dihedrals.sort_values(by=["atom1", "atom2", "atom3", "atom4"]).iterrows():
    mask = f"@{row['atom1']} @{row['atom2']} @{row['atom3']} @{row['atom4']}"
    gaff_string = str(pmd.tools.actions.printDihedrals(gaff_prmtop, mask))
    gaff_df = parse_parmed_output(gaff_string)
    
    smirnoff_string = str(pmd.tools.actions.printDihedrals(smirnoff_prmtop, mask))
    smirnoff_df = parse_parmed_output(smirnoff_string)
    
    if not len(gaff_df) == len(smirnoff_df):
        print(f"GAFF v1.7 has {len(gaff_df)} and SMIRNOFF99Frosst has {len(smirnoff_df)}")
        print(f"{row['atom1']}-{row['atom2']}-{row['atom3']}-{row['atom4']}")
    compare_df(gaff_df.sort_values(by=["atom1", "atom2", "atom3", "atom4", "periodicity", "phase"]), 
               smirnoff_df.sort_values(by=["atom1", "atom2", "atom3", "atom4", "periodicity", "phase"]))


Disparity in EEL scale.
Look at   C1 -   C2-  C3 -   C4
Disparity in VDW scale.
Look at   C1 -   C2-  C3 -   C4
Disparity in EEL scale.
Look at   C1 -   C2-  C3 -   C4
Disparity in VDW scale.
Look at   C1 -   C2-  C3 -   C4
Disparity in barrier height.
Look at   C1 -   C2-  C3 -   H3
Disparity in barrier height.
Look at   C1 -   C2-  C3 -   O3
Disparity in EEL scale.
Look at   C1 -   C2-  O2 -  HO2
Disparity in VDW scale.
Look at   C1 -   C2-  O2 -  HO2
Disparity in EEL scale.
Look at   C1 -   O5-  C5 -   C4
Disparity in VDW scale.
Look at   C1 -   O5-  C5 -   C4
Disparity in EEL scale.
Look at   C1 -   O5-  C5 -   C4
Disparity in VDW scale.
Look at   C1 -   O5-  C5 -   C4
Disparity in EEL scale.
Look at   C1 -   O5-  C5 -   C6
Disparity in VDW scale.
Look at   C1 -   O5-  C5 -   C6
Disparity in EEL scale.
Look at   C2 -   C1-  O5 -   C5
Disparity in VDW scale.
Look at   C2 -   C1-  O5 -   C5
Disparity in EEL scale.
Look at   C2 -   C1-  O5 -   C5
Disparity in VDW scale.
Look at   C2 -

In [74]:
def parse_parmed_output(printDihedrals_string):
    df = pd.DataFrame()
    lines = printDihedrals_string.split("\n")
    for line in lines[1:-1]:
        df = df.append(
            pd.DataFrame(
                {
                    "atom1": "".join(line[10:15]),
                    "atom1 serial": int("".join(line[5:10])),
                    "atom2": "".join(line[30:35]),
                    "atom2 serial": int("".join(line[25:30])),
                    "atom3": "".join(line[52:57]),
                    "atom3 serial": int("".join(line[47:52])),
                    "atom4": "".join(line[72:77]),
                    "atom4 serial": int("".join(line[67:72])),
                    "height": float("".join(line[89:95])),
                    "periodicity": float("".join(line[100:105])),
                    "phase": float("".join(line[111:116])),
                    "EEL scale": float("".join(line[122:128])),
                    "VDW scale": float("".join(line[133:138])),               },
                index=[0],
            ),
            ignore_index=True,
        )
    return df

In [133]:
def compare_df(gaff_df, smirnoff_df):
    for (_, gaff_row), (_, smirnoff_row) in zip(gaff_df.iterrows(), smirnoff_df.iterrows()):
        
        if gaff_row["height"] != smirnoff_row["height"]:
            print("Disparity in barrier height.")
            atoms = gaff_row["atom1"] + "-" + gaff_row["atom2"] + "-" + gaff_row["atom3"] + "-" + gaff_row["atom4"]
            print(f"Look at {atoms}")
#             gaff_string = gaff_row["atom1"] + "-" + \
#                           gaff_row["atom2"] + "-" + \
#                           gaff_row["atom3"] + "-" + \
#                           gaff_row["atom4"] + " " + str(gaff_row["height"])
            
#             smirnoff_string = smirnoff_row["atom1"] + "-" + \
#               smirnoff_row["atom2"] + "-" + \
#               smirnoff_row["atom3"] + "-" + \
#               smirnoff_row["atom4"] + " : " + str(smirnoff_row["height"])

#             print(gaff_string)
#             print(smirnoff_string)
        if gaff_row["periodicity"] != smirnoff_row["periodicity"]:
            print("Disparity in periodicity.")
            atoms = gaff_row["atom1"] + "-" + gaff_row["atom2"] + "-" + gaff_row["atom3"] + "-" + gaff_row["atom4"]
            print(f"Look at {atoms}")

        if gaff_row["phase"] != smirnoff_row["phase"]:
            print("Disparity in phase.")
            atoms = gaff_row["atom1"] + "-" + gaff_row["atom2"] + "-" + gaff_row["atom3"] + "-" + gaff_row["atom4"]
            print(f"Look at {atoms}")

        if gaff_row["EEL scale"] != smirnoff_row["EEL scale"]:
            print("Disparity in EEL scale.")
            atoms = gaff_row["atom1"] + "-" + gaff_row["atom2"] + "-" + gaff_row["atom3"] + "-" + gaff_row["atom4"]
            print(f"Look at {atoms}")

        if gaff_row["VDW scale"] != smirnoff_row["VDW scale"]:
            print("Disparity in VDW scale.")
            atoms = gaff_row["atom1"] + "-" + gaff_row["atom2"] + "-" + gaff_row["atom3"] + "-" + gaff_row["atom4"]
            print(f"Look at {atoms}")


In [136]:
gaff_df

,atom1,atom1 serial,atom2,atom2 serial,atom3,atom3 serial,atom4,atom4 serial,height,periodicity,phase,EEL scale,VDW scale
0,H1,2,C1,1,C2,4,O2,6,0.1556,3.0,0.0,1.2,2.0


In [137]:
smirnoff_df

,atom1,atom1 serial,atom2,atom2 serial,atom3,atom3 serial,atom4,atom4 serial,height,periodicity,phase,EEL scale,VDW scale
0,H1,2,C1,1,C2,4,O2,6,0.00,3.0,0.0,1.2,2.0
1,H1,2,C1,1,C2,4,O2,6,0.25,1.0,0.0,1.0,1.0
